# Second Administrative Divisions of Countries

**[Work in progress]**

This notebook creates a .csv file with second administrative divisions (Counties in the US) for ingestion into the Knowledge Graph.

Data source: [GeoNames.org](https://download.geonames.org/export/dump/)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
from pathlib import Path
import pandas as pd

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-19636412-9e74-4bac-8a4c-c6c8b49bb9d3/installation-4.1.0/import


### Create admin2

In [4]:
admin2_url = 'https://download.geonames.org/export/dump/admin2Codes.txt'

In [5]:
names = ['code', 'name', 'name_ascii', 'geonameid']

In [6]:
admin2 = pd.read_csv(admin2_url, sep='\t', dtype='str', names=names)
admin2 = admin2[['code', 'name_ascii', 'geonameid']]

### Standardize column names for Knowlege Graph
* id: unique identifier for country
* name: name of node
* parent_id: unique identifier for continent
* properties: camelCase

In [7]:
admin2.rename(columns={'code': 'id'}, inplace=True) # standard id column to link nodes
admin2.rename(columns={'name_ascii': 'name'}, inplace=True)
admin2.rename(columns={'geonameid': 'geonameId'}, inplace=True)
admin2['parentId'] = admin2['id'].str.rsplit('.', 1, expand=True)[0]

### Example

In [8]:
admin2.query("name == 'San Diego County'")

,id,name,geonameId,parentId
42196,US.CA.073,San Diego County,5391832,US.CA


### Export a minimum subset for now

In [9]:
admin2.head()

,id,name,geonameId,parentId
0,AE.01.101,Abu Dhabi Municipality,12047239,AE.01
1,AE.01.102,Al Ain Municipality,12047240,AE.01
2,AE.01.103,Al Dhafra,12047241,AE.01
3,AE.04.701,Al Fujairah Municipality,12047242,AE.04
4,AE.04.702,Dibba Al Fujairah Municipality,12047243,AE.04


In [10]:
admin2.to_csv(NEO4J_IMPORT / "00g-GeoNamesAdmin2.csv", index=False)